## Part 3: Learned shape embeddings

In this notebook, we will learn **...TODO...**.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

### 0. Data loading

**0.1** Run the lines below to load the object models dataset.

In [ ]:
# BBBC020
#directory = 'data/BBBC020/masks_cells'
directory = 'data/BBBC020/masks_nuclei'
point_models=np.load(os.path.join(directory,'point_models.npy'))